In [29]:
"""
This file provides example code for how to perform projections using the tomo3dt package.
"""

# Import code
import sys
import os 
import pickle
import numpy as np
import mrcz
sys.path.append('../')

import tomo3dt
from tomo3dt.algorithms import fp, sirt
from tomo3dt import operators

In [9]:
# initial save path and time for collection of each 3D structure
path =  os.path.join(os.getcwd(),'..', "test_data")
sim_path = os.path.join(path,"simulation.vmf")
vol_path = os.path.join(path,"volume.vmf")

sim = tomo3dt.VolumenDt(sim_path)

# Determine angle and time of collection
times = np.linspace(1, 71, 71)
gr = (1+np.sqrt(5))/2
alpha = np.radians(140)
angles = np.degrees((times*gr*alpha % alpha)-(alpha/2))

In [20]:
#TODO: Remove this section when done 
# File Corruption repare simulation .vmf file 
sim = tomo3dt.VolumenDt(sim_path)
emat_path = r"y:\TimC\misc\Simulations\SIM_SB_TC_004\vol"
for i in range(72):
    origin_path = os.path.join(emat_path,str(i)+"_vol_sim.rec")
    image, meta = mrcz.readMRC(origin_path)
    image = np.array(image).squeeze()
    target_size = (512, 512, 512)
    pad_width = [(max((target_size[i] - image.shape[i]) // 2, 0), 
              max((target_size[i] - image.shape[i] + 1) // 2, 0))
             for i in range(3)]
    obj = np.pad(image, pad_width, 'edge')
    sim.write_vmf_record(obj,i,i,i)


C:\Users\admin\AppData\Local\Temp\ipykernel_14996\401317362.py:7: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  image, meta = mrcz.readMRC(origin_path)
C:\Users\admin\AppData\Local\Temp\ipykernel_14996\401317362.py:7: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  image, meta = mrcz.readMRC(origin_path)
C:\Users\admin\AppData\Local\Temp\ipykernel_14996\401317362.py:7: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  image, meta = mrcz.readMRC(origin_path)
C:\Users\admin\AppData\Local\Temp\ipykernel_14996\401317362.py:7: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  image, meta = mrcz.readMRC(origin_path)
C:\Users\admin\AppData\Local\Temp\ipykernel_14996\401317362.py:7: De

In [21]:
# Perform Forward Projection
# This will provide the tiltseries where 1 projection is provided for each time in the volume time series
ts = operators.forwardproject(sim,angles)



In [ ]:
# Alternatively, we can perform a forward projection estimating approximating the tiltseries where each projection is collected at a specified time 
# so long as the number of times and angles are the same
ts = operators.forwardproject(sim,angles,times)

In [26]:
pickle.dump(ts,open(os.path.join(path,"tiltseries.pkl"),'wb'))

In [31]:
#Check no data exists in vol path to avoid error.
if os.path.isfile(vol_path):
    os.remove(vol_path)

# To performa a back projection specify the path to save the volume, the tiltseries, and the algorithm to use
# tomo3dt comes packaged with a fbp and sirt but you can specify your own algorithm
# Backprojections in tomo3dt work with frames and offsets. 
#   Frames are the number of projections to use for each reconstruction
#   Offsets are the number of projections to move the frame by between reconstructions 
# e.g. the example here shows a reconstruction where projections 1-20 are reconstructed, then 2-21, then 3-22, etc.
algor = lambda x: sirt(x, 100)
obj = operators.backproject(vol_path, ts, algor, 20, 1)

c:\Users\admin\anaconda3\envs\tomo3dt-env\Lib\site-packages\tomosipo\links\numpy.py:27: UserWarning: The parameter initial_value is of type float64; expected `np.float32`. The type has been Automatically converted. Use `ts.link(x.astype(np.float32))' to inhibit this warning. 
  warnings.warn(
c:\Users\admin\anaconda3\envs\tomo3dt-env\Lib\site-packages\tomosipo\links\numpy.py:27: UserWarning: The parameter initial_value is of type float64; expected `np.float32`. The type has been Automatically converted. Use `ts.link(x.astype(np.float32))' to inhibit this warning. 
  warnings.warn(
c:\Users\admin\anaconda3\envs\tomo3dt-env\Lib\site-packages\tomosipo\links\numpy.py:27: UserWarning: The parameter initial_value is of type float64; expected `np.float32`. The type has been Automatically converted. Use `ts.link(x.astype(np.float32))' to inhibit this warning. 
  warnings.warn(
c:\Users\admin\anaconda3\envs\tomo3dt-env\Lib\site-packages\tomosipo\links\numpy.py:27: UserWarning: The parameter ini

In [ ]:
#Check no data exists in vol path to avoid error.
if os.path.isfile(vol_path):
    os.remove(vol_path)

# Some cases exist whereby the number of projections for each reconstruction is not always the same
# particularly in situ fast tomography- where low quality projections are removed making the number of projections for each reconstruction variable
# you can specify the projections to use for each reconstruction with a tuple list
projections =  [(1,20),(20,40),(40,60),(60,71)]
algor = lambda x: tomo3dt.sirt(x, 100)
obj = operators.backproject(vol_path, ts, algor, proj_list= projections)